In [1]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd

#set ups
#pandas originally cut the data frames, this is used to desplay the full df
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


In [2]:
url = 'https://www.hellenicparliament.gr/Vouli-ton-Ellinon/To-Politevma/Ekloges/Eklogika-apotelesmata-New/'
response = requests.get(url)
response
#milisa sto server kai mou apanthse

<Response [200]>

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html lang="el" xml:lang="el" xmlns="http://www.w3.org/1999/xhtml">
<head id="ctl00_Head1"><title>
	Εκλογικά Αποτελέσματα
</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="text/css" http-equiv="Content-Style-Type"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/styles/default.css?v=1" media="screen" rel="Stylesheet" type="text/css"/>
<link href="/styles/inner.css" media="screen" rel="Stylesheet" type="text/css"/>
<link href="/styles/print.css" media="print" rel="Stylesheet" type="text/css"/>
<link href="/styles/jquery-ui.css" media="screen" rel="Stylesheet" type="text/css"/><script src="/scripts/jquery-ui/jquery.js" type="text/javascript"></script><script src="/scripts/jquery-ui.js" type="text/javascript"></script><scrip

In [4]:
#found the table
#create a variable called 'tables'
tables=soup.find_all('table')
#I want to find how many tables there are
len(soup.find_all('table'))

20

In [5]:
#be clean, create a variable called 'caption'
#find the caption content
caption = soup.find_all('caption')
#gives back, in list, all the captions
len(soup.find_all('caption'))

18

In [ ]:
data = [] 

for table in tables :
    rows = table.find_all('tr')#tr είναι η σειρά του πίνακα
    
    for row in rows :
        #print(row.text.strip())#για να μην πάρω html element
        cells = row.find_all('td')#td είναι τα κελιά
        #από τις τιμές όλων των κελιών, διαλέγω ποιο από τα 5 κελιά ...
        #...περιεχει αυτό που θέλω (από τα 5 κελιά, στο 1ο αρα στο 0)
        parties = cells[0::5]
        leaders = cells[1::5]
        seats = cells[2::5]
        
        #print (...)
           
        
        
    for cell in cells:
        #print(cell.text.strip())
        data.append(cell.text.strip())
        



In [ ]:
#put everything in one script!

#Step 1: create an empty list named data
data = []

#Step 2: find all tables on the webpage and store them into a variable named tables
tables = soup.find_all('table')

#Step 3: loop through your tables 
#for each table in tables
for table in tables:

    #find all rows
    rows = table.find_all('tr')[1:]
    
    #forw each row in rows
    for row in rows:
        
        #find all cell values
        cells = row.find_all('td')
        
        #define parties, leaders, seats, percentages and votes are found in your list of cell values 
        parties = cells[0::5]
        leaders = cells[1::5]
        seats = cells[2::5]
        percentages = cells[3::5]
        total_votes = cells[4::5]
        
        #Step 4: create an empty dictionary named mydata
        mydata = {}
        
        #We also have captions with useful info about parliamentary periods. Remember?
        #Grab the table caption and store it in a varialbe named caption
        caption = table.find('caption').text.strip()
        
        #create a dict key named caption and store your caption value there
        mydata['caption'] = caption
        
        #you have only one caption per table. But you have many parties in each table!
        #loop through your parties
        #for each party in parties
        for party in parties:
            #create a dict key named party and store the party value there. 
            #it gonna store all the party values, because you are looping through parties!
            mydata['party'] = party.text.strip()
    
        #do the same for leaders, seats, percentages and votes
        for leader in leaders:
            mydata['leader'] = leader.text.strip()
            
        for seat in seats:
            mydata['seats'] = seat.text.strip()
            
        for pct in percentages:
            mydata['pct'] = pct.text.strip()
            
        for votes in total_votes:
            mydata['votes'] = votes.text.strip()

        #Step 5: append your data list with whatever your mydata dictionary holds    
        data.append(mydata)

In [ ]:
len(data)

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df.party.value_counts()

In [ ]:
for i,item in df.iterrows():
    if item['leader'] == 'Σαμαράς Αντώνιος του Κωνσταντίνου': 
        df.at[i,'party']= 'Νέα Δημορκατία'
df

In [ ]:
df = df.dropna(thresh=3)


In [ ]:
df.caption[0]

In [ ]:
df['caption']= df.caption.str.replace('\r\n\tΙΗ','')
df

In [ ]:
df['elections_day'] = df.caption.str.extract(r'(\d+\/\d+\/\d+)')
df['start_date'] = df.caption.str.extract(r"(\d+\/\d+\/\d+).*\d+\/\d+\/\d+$")
df['end_date'] = df.caption.str.extract(r"(\d+\/\d+\/\d+)$")